In [1]:
import pandas as pd
import re
import multiprocessing

In [2]:
cpa_train_gt = pd.read_csv('SOTAB CPA/sotab_v2_cpa_training_set.csv')
cpa_test_gt = pd.read_csv('SOTAB CPA/sample_test.csv')

In [ ]:
cpa_train_gt

In [ ]:
cpa_test_gt

In [6]:
cpa_test_gt["class"] = cpa_test_gt["table_name"].apply(lambda x: x.split("_")[0])

In [6]:
cpa_test_gt

,table_name,main_column_index,column_index,label,class
0,Book_antipodean.com_September2020_CPA.json.gz,0,4,publisher,Book
1,Event_bfodurham.net_September2020_CPA.json.gz,0,3,eventStatus,Event
2,Event_bfodurham.net_September2020_CPA.json.gz,0,4,eventAttendanceMode,Event
3,Event_healthychelsea.org_September2020_CPA.jso...,0,4,organizer,Event
4,Event_healthychelsea.org_September2020_CPA.jso...,0,5,telephone,Event
...,...,...,...,...,...
504,Product_cit.li_September2020_CPA.json.gz,0,2,weight,Product
505,Product_cit.li_September2020_CPA.json.gz,0,4,productID,Product
506,Product_coininvest.com_September2020_CPA.json.gz,0,4,manufacturer,Product
507,Recipe_bakedbyanintrovert.com_September2020_CP...,0,11,recipeInstructions,Recipe


In [7]:
gt = {'train':{}, 'test':{}}
for index, row in cpa_train_gt.iterrows():
    if row["table_name"] not in gt['train']:
        gt['train'][row["table_name"]] = {}
    gt['train'][row["table_name"]][row["column_index"]] = row["label"]
    
for index, row in cpa_test_gt.iterrows():
    if row["table_name"] not in gt['test']:
        gt['test'][row["table_name"]] = {}
    gt['test'][row["table_name"]][row["column_index"]] = row["label"]

In [8]:
def clean_text(text):
        
    if(isinstance(text, dict)):
        text = ' '.join([ clean_text(v) for k, v in text.items()] )
    elif(isinstance(text, list)):
        text = map(clean_text, text)
        text = ' '.join(text)
        
    if pd.isnull(text):
        return ''
    
    text = re.sub(' +', ' ', str(text)).strip()
    
    return text

In [11]:

def get_all_table_columns(file_name, index):
    
    if file_name in cpa_train_gt["table_name"].tolist():
        path = 'SOTAB CPA/Train/'+file_name
    else:
        path = 'SOTAB CPA/Test/'+file_name
    
    df = pd.read_json(path, compression='gzip', lines=True)
        
    cleaned_rows = []

    cleaned_main="{}{}"

    x = " ".join([" ".join(clean_text(row).split()[:20]) for row in df[0].tolist()[:5]])
    cleaned_main=cleaned_main.format("Column 1: ", x)
    cleaned_rows.append(cleaned_main)

    cleaned_rows.append("\n Column 2:")
    
    
    for row in df.iloc[:, index].tolist():
        get_values_var=7
        cleaned = " ".join(clean_text(row).split()[:20])
        
        if cleaned != "":
            cleaned_rows.append(cleaned)
        else:
            get_values_var=get_values_var-1

    
    return " ".join(cleaned_rows[:get_values_var]) 

In [12]:
test_examples = []
for table in gt['test']:
    for column in gt['test'][table]:
        col_str = get_all_table_columns(table, column)
        test_examples.append([table, column, col_str, gt['test'][table][column], table.split("_")[0]])

In [ ]:
test_examples

In [ ]:
train_examples = []
for table in gt['train']:
    for column in gt['train'][table]:
        col_str = get_all_table_columns(table, column)
        train_examples.append([table, column, col_str, gt['train'][table][column], table.split("_")[0]])

In [ ]:
train_examples

In [ ]:
len(train_examples)

In [14]:
import pickle

In [ ]:
file_name='SOTAB CPA/sotabv2-cpa-train-column.pkl'
f = open(file_name,'wb')
pickle.dump(train_examples,f)
f.close()

In [15]:
file_name='SOTAB CPA/sotabv2-cpa-sample-test-column.pkl'
f = open(file_name,'wb')
pickle.dump(test_examples,f)
f.close()